# Numerično integriranje

**Datum**: 10/11/2024

**Avtor**: Aleksander Grm

V zapiskih so uporabljeni primeri iz OnLine knjige [Numerične metode v ekosistemu Pythona, Janko Slavič](https://jankoslavic.github.io/pynm)

<hr>

Najprej naložimo celoten potreben Python ekosistem

In [ ]:
import numpy as np              # orodja za numeriko
import matplotlib.pyplot as mpl # izdelava grafov
import numpy.polynomial as poly # paket za podporo polinomov
import scipy.optimize as opt    # uporaba fsolve() funkcije

from IPython.display import YouTubeVideo